# 連接雲端與切換資料夾

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/專題實驗/training_3-15/

/content/drive/MyDrive/專題實驗/training_3-15


# 訓練與測試

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

imgSize = (224, 224)
batchSize = 32

# set the training dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "train",
    validation_split = 0.1,
    subset = "training",
    seed = 1337,
    image_size = imgSize,
    batch_size = batchSize,
)

# set the validation dataset
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "validation",
    validation_split = 0.9,
    subset = "validation",
    seed = 1337,
    image_size = imgSize,
    batch_size = batchSize,
)

# set the data augmentation
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.Rescaling(0.7),
        layers.Rescaling(0.8),
        layers.RandomTranslation((-0.2, 0.3), (-0.2, 0.3)),
    ]
)

# set the training model
def setModel(inputSize, typeNum):
    input = keras.Input(shape = inputSize)
    x = data_augmentation(input)
    x = layers.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    previous_block_activation = x

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])
        previous_block_activation = x

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.GlobalAveragePooling2D()(x)
    activation = "sigmoid"
    units = 1

    x = layers.Dropout(0.2)(x)
    output = layers.Dense(units, activation=activation)(x)
    return keras.Model(input, output)


model = setModel(imgSize + (3,), 2)
keras.utils.plot_model(model, show_shapes=True)

epochs = 50
callbacks = [keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

model.fit(
    train_ds,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = val_ds,
)

# save the final model and weights
model.save('model.hdf5')
model.save_weights('model_weights.h5')

Found 2712 files belonging to 2 classes.
Using 2441 files for training.
Found 793 files belonging to 2 classes.
Using 713 files for validation.
Epoch 1/50
77/77 [==============================] - 868s 11s/step - loss: 0.5325 - accuracy: 0.7460 - val_loss: 0.7096 - val_accuracy: 0.5091
Epoch 2/50
77/77 [==============================] - 32s 407ms/step - loss: 0.3958 - accuracy: 0.8197 - val_loss: 0.7939 - val_accuracy: 0.5091
Epoch 3/50
77/77 [==============================] - 32s 407ms/step - loss: 0.3601 - accuracy: 0.8443 - val_loss: 1.0059 - val_accuracy: 0.5091
Epoch 4/50
77/77 [==============================] - 32s 406ms/step - loss: 0.3198 - accuracy: 0.8636 - val_loss: 0.9739 - val_accuracy: 0.5091
Epoch 5/50
77/77 [==============================] - 32s 405ms/step - loss: 0.2988 - accuracy: 0.8730 - val_loss: 0.9320 - val_accuracy: 0.5091
Epoch 6/50
77/77 [==============================] - 32s 405ms/step - loss: 0.2744 - accuracy: 0.8816 - val_loss: 0.6599 - val_accuracy: 0.5568

# 預測

In [4]:
%cd /content/drive/MyDrive/執行結果/3-17執行結果_2/
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import load_img

# load the model
model = load_model('model.hdf5')

# load the image
%cd /content/drive/MyDrive/專題實驗/test/
img = keras.preprocessing.image.load_img(
    "110.jpg", target_size = (224, 224)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

# predict and get the result
predictions = model.predict(img_array)
score = predictions[0]
print(
    "This image is %.2f percent rainy and %.2f percent sunny."
    % (100 * (1 - score), 100 * score)
)

/content/drive/MyDrive/執行結果/3-17執行結果_2
/content/drive/MyDrive/專題實驗/test
This image is 2.43 percent rainy and 97.57 percent sunny.
